<a href="https://colab.research.google.com/github/fractalsproject/spacenet6challenge/blob/master/notebooks/SpaceNet6Challenge_Baseline_TrainSAROnly_PyTorch_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#
# Training config
#
train = False

# If trying to resume a training session, this must be the name of the dated directory
resume = None

# Set to True if you are debugging Solaris at the backend
reinstall_solaris = False

#
# Test config
#
test = False

# Top level director of the weights to load.  Set to None to use from the training session above.
read_model_weights_toplevel_dir = None #'./spacenet6challenge/CosmiQ_SN6_Baseline'

In [2]:
# Clone the spacenet6challenge project
!rm -r spacenet6challenge
!if [ ! -d "spacenet6challenge" ]; then git clone --recursive "https://github.com/fractalsproject/spacenet6challenge.git" ; else echo "spacenet6challenge directory already exists"; fi

Cloning into 'spacenet6challenge'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 503 (delta 110), reused 117 (delta 49), pack-reused 319
Receiving objects: 100% (503/503), 130.00 KiB | 355.00 KiB/s, done.
Resolving deltas: 100% (242/242), done.
Submodule 'CosmiQ_SN6_Baseline' (https://github.com/CosmiQ/CosmiQ_SN6_Baseline/) registered for path 'CosmiQ_SN6_Baseline'
Submodule 'solaris' (https://github.com/fractalsproject/solaris.git) registered for path 'solaris'
Cloning into '/content/spacenet6challenge/CosmiQ_SN6_Baseline'...
remote: Enumerating objects: 323, done.        
remote: Total 323 (delta 0), reused 0 (delta 0), pack-reused 323        
Receiving objects: 100% (323/323), 81.24 MiB | 11.27 MiB/s, done.
Resolving deltas: 100% (187/187), done.
Cloning into '/content/spacenet6challenge/solaris'...
remote: Enumerating objects: 5, done.        
remote: Counting objects: 100% 

In [3]:
# Check if we need to do a full installation
try:
  import solaris
  from solaris import utils
  print("Found solaris package.  Assuming a previous installation worked.")
  solaris_ok = True
except:
  import sys
  sys.path.append('/content/spacenet6challenge')
  import spacenet6.colab.setup
  spacenet6.colab.setup.baseline_prereqs(force=True)
  solaris_ok = False

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Found solaris package.  Assuming a previous installation worked.


In [0]:
# Useful for print debugging solaris
reinstall_solaris = False
if reinstall_solaris:
  !pip uninstall solaris
  !cd /content/spacenet6challenge/solaris && python setup.py clean
  !cd /content/spacenet6challenge/solaris && pip install .
  import solaris
  import importlib
  importlib.reload(solaris)
  solaris_ok = True

In [0]:
# TODO:  Why do we need to do this here...
if not solaris_ok:
  !pip install shapely
  !cd /content/spacenet6challenge/solaris && python setup.py clean
  !cd /content/spacenet6challenge/solaris && pip install .
  try:
    #from solaris import utils
    #print("Found solaris package.  Assuming installation worked.")
    solaris_ok = True
  except:
    raise Exception("You should restart the kernel and run the notebook again.")

In [6]:
import solaris
dir(solaris)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'bin',
 'data',
 'eval',
 'nets',
 'raster',
 'tile',
 'utils',
 'vector']

In [7]:
import importlib
importlib.reload(solaris)
import solaris
dir(solaris)


['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'bin',
 'data',
 'eval',
 'nets',
 'raster',
 'tile',
 'utils',
 'vector']

In [8]:
# Check tensorflow 
import tensorflow
print("tensorflow version=",tensorflow.version.VERSION)
if (tensorflow.version.VERSION!='1.13.1'):
  raise Exception("You need to restart the kernel and resume from here.")

# Check torch 
import torch
print( "torch version=", torch.version.__version__) 
if not torch.version.__version__.startswith("1.5"):
  raise Exception("You need restart the kernel and resume from here.")
if not torch.cuda.is_available():
  raise Exception("Torch cuda is not available.")
else:
  print("Torch cuda is available.")

# Check all installation packages are available
try:
  import solaris
  from solaris import utils
  import sys
  !pip install geopandas>=0.7.0
  import geopandas
  sys.path.append('/content/spacenet6challenge/CosmiQ_SN6_Baseline')
  import baseline
  sys.path.append('/content/spacenet6challenge')
  from spacenet6.cosmiq import baseline_wrap
except:
  raise Exception("Installation checks failed.")

print("Installation checks passed.")

tensorflow version= 1.13.1
torch version= 1.5.0+cu101
Torch cuda is available.
Installation checks passed.


In [9]:
# Mount GCP bucket data
import sys
sys.path.append('./spacenet6challenge')
import spacenet6.colab.bucket
spacenet6.colab.bucket.mount("spacenet_challenge_data",force_new_mount=True)

Checking for jupyter/ipython environment...
Passed.
Checking for colab environment...
Passed.
Trying to authenticate GCP user...
Running command: "echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list".  Please wait...
OK.
Running command: "curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add - ".  Please wait...
OK.
Running command: "apt -qq update".  Please wait...
OK.
Running command: "apt -qq install gcsfuse".  Please wait...
OK.
Running command: "mkdir mountOnColab".  Please wait...
OK.
Running command: "gcsfuse --implicit-dirs spacenet_challenge_data mountOnColab".  Please wait...
OK.
Done. Getting folder contents...
Running command: "ls -als /content/mountOnColab".  Please wait...

Mount was successful.


True

In [0]:
# Train on SAR only
import os
import datetime
import solaris as sol
import model

# validate pretrain path
pretrain_path = '/content/mountOnColab/pretrain'
if not os.path.exists(pretrain_path):
  raise Exception("pretrain path does not exist.")

# validate sessions toplevel
train_sessions_path = '/content/mountOnColab/train_sessions'
if not os.path.exists(train_sessions_path):
  raise Exception("train sessions path does not exist.")

# validate configs dir
configs_path = './spacenet6challenge/configs'
if not os.path.exists(configs_path):
  raise Exception("configs path does not exist.")

# create a dir for this training session
dpath = str(datetime.datetime.now()).replace(":","_").replace(" ","_")
outpath = '%s/%s' % ( train_sessions_path, dpath )
os.makedirs(outpath,exist_ok=True)
weightspath = os.path.join(outpath, "weights")
os.makedirs(weightspath,exist_ok=True)

# create train.csv from template
f = open( os.path.join(configs_path,'train.csv') )
txt = f.read()
f.close()
newtxt = txt.replace('./root',pretrain_path)
traincsv = os.path.join( outpath, "train.csv")
f = open(traincsv, 'w')
f.write(newtxt)
f.flush()
f.close()

# create valid.csv from template
f = open(os.path.join(configs_path,'valid.csv') )
txt = f.read()
f.close()
newtxt = txt.replace('./root',pretrain_path)
validcsv = os.path.join( outpath, "valid.csv")
f = open(validcsv, 'w')
f.write(newtxt)
f.flush()
f.close()

# create a SAR training yaml from template
f = open( os.path.join(configs_path, 'train_sar.yaml') )
txt = f.read()
f.close()
newtxt = txt.replace("DATEDIR",outpath)
newtxt = newtxt.replace("TRAINCSV",traincsv)
newtxt = newtxt.replace("VALIDCSV",validcsv)
newyaml = os.path.join( outpath, "train_sar.yaml")
f = open(newyaml,'w')
f.write(newtxt)
f.flush()
f.close()

print("Wrote file->%s" % newyaml)

# create a training config from yaml
config = sol.utils.config.parse(newyaml)
config['pretrained'] = False

sar_dict = {
    'model_name': 'unet11',
    'weight_path': None,
    'weight_url': None,
    'arch': model.UNet11
}

# train
trainer = sol.nets.train.Trainer(config, custom_model_dict=sar_dict)
trainer.train()

Wrote file->/content/mountOnColab/train_sessions/2020-05-06_15_34_04.994493/train_sar.yaml
Beginning training epoch 0
Train Pytorch CUDA AVAIL


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


    loss at batch 0: 2.5997495651245117
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
    loss at batch 10: 1.3390947580337524
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
    loss at batch 20: 1.109626054763794
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
    loss at batch 30: 1.0514026880264282
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch CUDA AVAIL
Train Pytorch

In [0]:
if train:

  # Verify all the directories
  import os
  import datetime

  train_path = "/content/mountOnColab/SAR/train/AOI_11_Rotterdam"
  if not os.path.exists(train_path):
    raise Exception("Could not find training data")

  pretrain_path = '/content/mountOnColab/pretrain'
  if not os.path.exists(pretrain_path):
    raise Exception('Could not find pretrain data')

  train_sessions_path = '/content/mountOnColab/train_sessions'

  # are we trying to resume a training session 
  if resume: 

    # Get the last trained model and figure out where to resume...
    print("Will try to resume training from trainining=>", resume)
    dpath = resume
    outpath = '%s/%s' % ( train_sessions_path, dpath )
    if not os.path.exists( outpath ):
      raise Exception("Can't resume because %s does not exist." % outpath)
    subdirs = os.listdir( os.path.join(outpath, "weights"))
    print("listdir=", subdirs)
    epochs = sorted( [ int( subdir.split(".")[0] ) for subdir in subdirs if subdir.split(".")[0].isnumeric() ] )
    print("all epochs=", epochs)
    resume_epoch = int(epochs[-1])
    resume_weights = os.path.join(outpath,"weights/%d.hdf5" % resume_epoch )
    raise Exception("Not finished.")
  else:

    # Create new dated directory for saved weights...
    resume_epoch = None
    resume_weights = None
    dpath = str(datetime.datetime.now()).replace(":","_").replace(" ","_")
    outpath = '%s/%s' % ( train_sessions_path, dpath )

  os.makedirs(outpath, exist_ok=True)
  print("model output directory=", outpath)

  weights_dir = os.path.join(outpath,"weights")
  os.makedirs( weights_dir, exist_ok=True)
  print("model weights directory=", weights_dir)

  # Start the training session

  cmdargs = ["--train",
                  "--sardir","%s/SAR-Intensity" % train_path,
                  "--opticaldir","%s/PS-RGB" % train_path,
                  "--labeldir","%s/geojson_buildings" % train_path,
                  "--rotationfile","%s/SummaryData/SAR_orientations.txt" % train_path,
                             "--rotationfilelocal","%s/SAR_orientations.txt" % pretrain_path,
                              "--maskdir", "%s/masks" % pretrain_path,
                              "--sarprocdir", "%s/sartrain" % pretrain_path, 
                              "--opticalprocdir","%s/optical" % pretrain_path,
                              "--traincsv","%s/train.csv" % pretrain_path,
                              "--validcsv", "%s/valid.csv" % pretrain_path,
                              "--opticaltraincsv", "%s/opticaltrain.csv" % pretrain_path,
                              "--opticalvalidcsv", "%s/opticalvalid.csv" % pretrain_path,
                              "--testcsv","%s/test.csv" % pretrain_path,
                              "--yamlpath","%s/trainsar.yaml" % outpath,
                              "--opticalyamlpath","%s/optical.yaml" % outpath,
                              "--modeldir","%s/weights" % outpath,
                              "--testprocdir","%s/sartest" % outpath,
                              "--testoutdir","%s/inference_continuous" % outpath,
                              "--testbinarydir","%s/inference_binary" % outpath,
                              "--testvectordir","%s/inference_vectors" % outpath,
                              "--rotate",
                              "--transferoptical",
                              "--mintrainsize","20",
                              "--mintestsize","80"]
  args = baseline_wrap.parse_args(cmdargs)
  #print(args)
  #import importlib
  #importlib.reload(baseline_wrap)
  baseline_wrap.invoke(outpath, "/content/mountOnColab/pretrain", args)

In [0]:
# Evaluate on test data

if test:

  #model output directory= /content/mountOnColab/train_sessions/2020-05-03_14_04_02.011290
  #model weights directory= /content/mountOnColab/train_sessions/2020-05-03_14_04_02.011290/weights

  # Verify all the directories
  import os
  import datetime
  import importlib
  import sys
  sys.path.append('./spacenet6challenge/CosmiQ_SN6_Baseline')
  sys.path.append('./spacenet6challenge/')
  from spacenet6.cosmiq import baseline_wrap
  import baseline
  importlib.reload(baseline)
  import importlib
  importlib.reload(baseline_wrap)

  # where is the raw test SAR data?
  test_path = "/content/mountOnColab/SAR/test_public/AOI_11_Rotterdam"
  if not os.path.exists(test_path):
    raise Exception("Could not find training data")

  # where is the pretrain_pretest data?
  pretrain_pretest_path = '/content/mountOnColab/pretrain'
  if not os.path.exists(pretrain_pretest_path):
    raise Exception('Could not find pretrain data')

  # validate the directory where we load the weights.
  if not os.path.exists( os.path.join(read_model_weights_toplevel_dir, "weights" ) ):
    raise Exception("model weights dir does not exist.")

  # where to put output from this test eval session?
  test_sessions_path = '/content/mountOnColab/tests_sessions'
  dpath = str(datetime.datetime.now()).replace(":","_").replace(" ","_")
  outpath = '%s/%s' % ( test_sessions_path, dpath )
  os.makedirs( os.path.join( outpath, "sartest"), exist_ok=True )
  os.makedirs( os.path.join( outpath, "inference_continuous"), exist_ok=True )
  os.makedirs( os.path.join( outpath, "inference_binary"), exist_ok=True )
  os.makedirs( os.path.join( outpath, "inference_vectors"), exist_ok=True )

  cmdargs = [ "--test",
                  "--testdir", "%s/SAR-Intensity" % test_path,
                              "--rotationfilelocal","%s/SAR_orientations.txt" % pretrain_pretest_path,
                              "--maskdir", "%s/masks" % pretrain_pretest_path,
                              "--sarprocdir", "%s/sartrain" % pretrain_pretest_path, 
                              "--opticalprocdir","%s/optical" % pretrain_pretest_path,
                              "--testcsv","%s/test.csv" % pretrain_pretest_path,
                              "--yamlpath","%s/infer.yaml" % pretrain_pretest_path,
                              "--modeldir","%s/weights" % read_model_weights_toplevel_dir,
                              "--testprocdir","%s/sartest" % outpath,
                              "--testoutdir","%s/inference_continuous" % outpath,
                              "--testbinarydir","%s/inference_binary" % outpath,
                              "--testvectordir","%s/inference_vectors" % outpath,
                              "--rotate",
                              "--mintestsize","80"]
  args = baseline_wrap.parse_args(cmdargs)
  #print(args)
  #import importlib
  #importlib.reload(baseline_wrap)
  print(vars(args))
  baseline_wrap.invoke(outpath, "/content/mountOnColab/pretrain", args)